In [1]:
from __future__ import print_function
import pyzbar.pyzbar as pyzbar
import numpy as np
import cv2
import yaml
import math
# 0.17m
length_ofedge=0.17
def decode(im,mtx,dist) : 
    # Find barcodes and QR codes
    
    decodedObjects = pyzbar.decode(im)

    # Print results
    for obj in decodedObjects:
        print('Type : ', obj.type)
        print('Data : ', obj.data,'\n')

    return decodedObjects

In [2]:
objp = np.array([[0., 0., 0.],
       [0., length_ofedge, 0.],
       [length_ofedge, length_ofedge, 0.],
       [length_ofedge, 0., 0.]])
with open('calibration.yaml') as f:
    loadeddict = yaml.load(f)

mtx = np.array(loadeddict.get('camera_matrix'))
dist = np.array(loadeddict.get('dist_coeff'))
axis = np.float32([[0.1,0,0], [0,0.1,0], [0,0,-0.1]]).reshape(-1,3)
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img
# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6
 
 
# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R) :
 
    assert(isRotationMatrix(R))
     
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([x, y, z])


In [3]:
objp

array([[0.  , 0.  , 0.  ],
       [0.  , 0.17, 0.  ],
       [0.17, 0.17, 0.  ],
       [0.17, 0.  , 0.  ]])

In [ ]:
# Display barcode and QR code location  
kk=None
def display(im, decodedObjects):
    dist=np.array([])
    # Loop over all decoded objects
    for decodedObject in decodedObjects: 
        points = decodedObject.location
        print (points)
        points=[point for point in points]
        # If the points do not form a quad, find convex hull
        if len(points) > 4 : 
            hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
            hull = list(map(tuple, np.squeeze(hull)))
        else : 
            hull = points;

        # Number of points in the convex hull
        n = len(hull)
        if any(h[0]>=2147483647 or h[1]>=2147483647 for h in hull):
            continue
        # Draw the convext hull
        for j in range(0,n):
            if hull[j][0]>=2147483647 or hull[j][1]>=2147483647 or hull[(j+1) % n][0]>=2147483647 or hull[(j+1) % n][1]>=2147483647:
                continue
            cv2.line(im, hull[j], hull[ (j+1) % n], (255,0,0), 3)

        font = cv2.FONT_HERSHEY_SIMPLEX
        for h in hull:
#             if h[0]>=2147483647 or h[1]>=2147483647:
#                 continue
            cv2.putText(im,str(h),h, font, 1,(255,255,255),2,cv2.LINE_AA)
        ret,rvecs, tvecs = cv2.solvePnP(objp, np.array(hull).astype(np.float32), mtx, dist)
        Rt = np.zeros(shape=(3,3))
        cv2.Rodrigues(rvecs,Rt)
        
#         t = (math.asin(-Rt[0][2]))

#         print (Rt)
#         print (tvecs)
        R = Rt.transpose()
        euler_angles=rotationMatrixToEulerAngles(R)
        pos = -R.dot(tvecs)
        cv2.putText(im,'Position of camera: '+str(pos),(50,50), font, 0.5,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(im,'Euler angles of object: '+str([euler_angle*180/math.pi for euler_angle in euler_angles][1]),(50,100), font, 0.5,(255,255,255),2,cv2.LINE_AA)
#         print (pos)
        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
        print (imgpts)
        im = draw(im,np.array(hull).astype(np.float32),imgpts)
    # Display results 
    cv2.imshow("Results", im);
    cv2.waitKey(1);
 
     

In [ ]:
# Main 
cap = cv2.VideoCapture(1)

while True:
    _, frame = cap.read()
    # Read image 
    frame = cv2.undistort(frame, mtx, dist)
    decodedObjects = decode(frame,mtx,dist)
    display(frame, decodedObjects)

    

Type :  QRCODE
Data :  GOAL 

[(32L, 11L), (11L, 202L), (217L, 204L), (232L, 7L)]
[[[147.79654    8.502278]]

 [[ 19.978899 120.91274 ]]

 [[-14.993536 -63.653564]]]
Type :  QRCODE
Data :  GOAL 

[(192L, 110L), (203L, 221L), (307L, 213L), (302L, 103L)]
[[[257.26703 105.86036]]

 [[198.23975 176.29193]]

 [[198.3771  108.00388]]]
Type :  QRCODE
Data :  GOAL 

[(192L, 110L), (203L, 220L), (306L, 213L), (301L, 102L)]
[[[255.26862 105.5339 ]]

 [[198.15326 175.18643]]

 [[176.64229 112.82835]]]
Type :  QRCODE
Data :  GOAL 

[(193L, 110L), (203L, 219L), (306L, 211L), (300L, 102L)]
[[[256.4336   105.43169 ]]

 [[198.69409  174.90504 ]]

 [[199.44946  106.519516]]]
Type :  QRCODE
Data :  GOAL 

[(194L, 110L), (204L, 218L), (307L, 211L), (301L, 103L)]
[[[256.62943  106.005295]]

 [[199.69644  174.29482 ]]

 [[183.28682  114.19203 ]]]
Type :  QRCODE
Data :  GOAL 

[(194L, 110L), (204L, 219L), (307L, 209L), (300L, 103L)]
[[[256.74274 105.62896]]

 [[200.02238 174.62532]]

 [[201.04892 104.96167]

Type :  QRCODE
Data :  GOAL 

[(219L, 119L), (222L, 219L), (304L, 209L), (303L, 97L)]
[[[266.71387 106.47688]]

 [[220.72876 177.8531 ]]

 [[177.79024 106.05973]]]
Type :  QRCODE
Data :  GOAL 

[(218L, 119L), (222L, 220L), (304L, 209L), (303L, 97L)]
[[[266.23898 106.47231]]

 [[220.25735 178.36368]]

 [[176.92897 106.39846]]]
Type :  QRCODE
Data :  GOAL 

[(218L, 119L), (222L, 220L), (304L, 209L), (303L, 97L)]
[[[266.23898 106.47231]]

 [[220.25735 178.36368]]

 [[176.92897 106.39846]]]
Type :  QRCODE
Data :  GOAL 

[(218L, 119L), (221L, 219L), (303L, 209L), (302L, 97L)]
[[[265.7158   106.476456]]

 [[219.72896  177.85307 ]]

 [[176.78235  106.034355]]]
Type :  QRCODE
Data :  GOAL 

[(209L, 117L), (221L, 220L), (288L, 209L), (279L, 90L)]
[[[248.35239 101.77839]]

 [[215.95485 177.86604]]

 [[157.89806 112.93254]]]
Type :  QRCODE
Data :  GOAL 

[(214L, 117L), (222L, 220L), (291L, 208L), (283L, 91L)]
[[[252.90762 102.32931]]

 [[218.7173  177.76065]]

 [[162.95915 111.41433]]]
Type :  QR

Type :  QRCODE
Data :  GOAL 

[(208L, 108L), (212L, 211L), (313L, 207L), (310L, 100L)]
[[[267.3242   103.31102 ]]

 [[210.42273  168.87389 ]]

 [[191.51193   98.250114]]]
Type :  QRCODE
Data :  GOAL 

[(205L, 105L), (210L, 210L), (312L, 207L), (308L, 103L)]
[[[265.94305  103.8074  ]]

 [[207.92207  166.89731 ]]

 [[210.49394   90.628716]]]
Type :  QRCODE
Data :  GOAL 

[(206L, 102L), (211L, 210L), (313L, 209L), (312L, 101L)]
[[[267.79007 101.47447]]

 [[208.76491 166.30122]]

 [[190.08495 107.3234 ]]]
Type :  QRCODE
Data :  GOAL 

[(205L, 107L), (211L, 217L), (310L, 212L), (309L, 106L)]
[[[266.93564  106.28181 ]]

 [[208.43599  172.3289  ]]

 [[221.72153  101.960205]]]
Type :  QRCODE
Data :  GOAL 

[(204L, 109L), (211L, 217L), (309L, 211L), (306L, 107L)]
[[[264.7018  107.66868]]

 [[208.08656 173.01817]]

 [[219.62653 102.49097]]]
Type :  QRCODE
Data :  GOAL 

[(203L, 110L), (210L, 217L), (309L, 211L), (305L, 107L)]
[[[263.61597 108.13686]]

 [[207.10695 173.38193]]

 [[215.72462 102.7

Type :  QRCODE
Data :  GOAL 

[(202L, 118L), (206L, 212L), (301L, 210L), (297L, 118L)]
[[[258.0369  117.891  ]]

 [[204.35414 172.90852]]

 [[205.81615  96.94606]]]
Type :  QRCODE
Data :  GOAL 

[(202L, 117L), (206L, 213L), (301L, 211L), (297L, 117L)]
[[[258.14664 116.92393]]

 [[204.3626  173.24629]]

 [[207.36845  99.1753 ]]]
Type :  QRCODE
Data :  GOAL 

[(202L, 117L), (206L, 213L), (301L, 210L), (298L, 117L)]
[[[258.5224  116.79154]]

 [[204.40062 173.12508]]

 [[205.36261  98.93668]]]
Type :  QRCODE
Data :  GOAL 

[(203L, 117L), (206L, 213L), (301L, 211L), (297L, 117L)]
[[[258.5719   116.974045]]

 [[204.78532  173.2475  ]]

 [[207.69556  100.33632 ]]]
Type :  QRCODE
Data :  GOAL 

[(203L, 117L), (206L, 213L), (301L, 211L), (298L, 117L)]
[[[259.0522   116.908936]]

 [[204.79276  173.25397 ]]

 [[206.55124  100.402794]]]
Type :  QRCODE
Data :  GOAL 

[(202L, 116L), (206L, 213L), (301L, 211L), (298L, 115L)]
[[[258.72345  115.354164]]

 [[204.34933  173.08463 ]]

 [[206.39755  101.68

Type :  QRCODE
Data :  GOAL 

[(214L, 98L), (212L, 205L), (314L, 209L), (319L, 105L)]
[[[276.09015 101.87643]]

 [[212.87627 161.13632]]

 [[221.76997  85.50903]]]
Type :  QRCODE
Data :  GOAL 

[(214L, 98L), (212L, 205L), (314L, 209L), (319L, 104L)]
[[[276.13687  101.3824  ]]

 [[212.81775  161.31598 ]]

 [[220.86856   87.690506]]]
Type :  QRCODE
Data :  GOAL 

[(214L, 97L), (212L, 205L), (314L, 210L), (319L, 104L)]
[[[276.29858  101.07841 ]]

 [[212.74188  160.86874 ]]

 [[224.45528   85.099464]]]
Type :  QRCODE
Data :  GOAL 

[(214L, 97L), (212L, 205L), (314L, 210L), (320L, 104L)]
[[[276.83255  100.992714]]

 [[212.7439   161.00285 ]]

 [[222.8182    86.77119 ]]]
Type :  QRCODE
Data :  GOAL 

[(214L, 97L), (212L, 205L), (314L, 210L), (319L, 104L)]
[[[276.29858  101.07841 ]]

 [[212.74188  160.86874 ]]

 [[224.45528   85.099464]]]
Type :  QRCODE
Data :  GOAL 

[(214L, 97L), (212L, 205L), (314L, 209L), (320L, 104L)]
[[[276.75296 100.84781]]

 [[212.84718 160.98253]]

 [[221.81538  87.9

In [ ]:
np.array(hull)




In [ ]:
objp

In [ ]:
print(np.array([[h] for h in hull]))